# Install the dependencies

In [2]:
!pip install tensorflow opencv-python-headless pillow numpy --user

  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


# Gather images

In [5]:
# Run this on you computer idle, then copy the folders to this enviornment
import cv2

name = 'Andy' #replace with your name

cam = cv2.VideoCapture(0)

cv2.namedWindow("press space to take a photo", cv2.WINDOW_NORMAL)
cv2.resizeWindow("press space to take a photo", 500, 500)

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("press space to take a photo", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = name +"/image_{}.jpg".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

Andy/image_0.jpg written!
Andy/image_1.jpg written!
Andy/image_2.jpg written!
Andy/image_3.jpg written!
Andy/image_4.jpg written!
Andy/image_5.jpg written!
Andy/image_6.jpg written!
Andy/image_7.jpg written!
Andy/image_8.jpg written!
Andy/image_9.jpg written!
Andy/image_10.jpg written!
Andy/image_11.jpg written!
Andy/image_12.jpg written!
Andy/image_13.jpg written!
Andy/image_14.jpg written!
Andy/image_15.jpg written!
Andy/image_16.jpg written!
Andy/image_17.jpg written!
Andy/image_18.jpg written!
Andy/image_19.jpg written!
Andy/image_20.jpg written!
Andy/image_21.jpg written!
Andy/image_22.jpg written!
Andy/image_23.jpg written!
Andy/image_24.jpg written!
Andy/image_25.jpg written!
Andy/image_26.jpg written!
Andy/image_27.jpg written!
Andy/image_28.jpg written!
Andy/image_29.jpg written!
Andy/image_30.jpg written!
Andy/image_31.jpg written!
Andy/image_32.jpg written!
Andy/image_33.jpg written!
Andy/image_34.jpg written!
Andy/image_35.jpg written!
Andy/image_36.jpg written!
Andy/image_

# Process the images

In [10]:
import cv2
import os
import numpy as np

def preprocess_image(image_path, target_size=(224, 224)):
    print(f"Loading image from: {image_path}")
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Error: Image at path {image_path} could not be loaded.")
        return None
    
    image = cv2.resize(image, target_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def load_dataset(dataset_path):
    images = []
    labels = []
    label_map = {}
    label_counter = 0
    
    for label_name in os.listdir(dataset_path):
        label_dir = os.path.join(dataset_path, label_name)
        if os.path.isdir(label_dir):
            if label_name not in label_map:
                label_map[label_name] = label_counter
                label_counter += 1
                
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image = preprocess_image(image_path)
                
                if image is not None:  # Only add the image if it loaded successfully
                    images.append(image)
                    labels.append(label_map[label_name])
    
    return np.array(images), np.array(labels), label_map


In [11]:
import os
import cv2

def validate_images(dataset_path):
    for label_name in os.listdir(dataset_path):
        label_dir = os.path.join(dataset_path, label_name)
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Invalid image: {image_path}")

validate_images('dataset')

Invalid image: dataset\Andy\.ipynb_checkpoints


# Build and Train Your Face Recognition Model

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(input_shape=(224, 224, 3), num_classes=2):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [13]:
images, labels, label_map = load_dataset('dataset')

model = create_model(input_shape=images.shape[1:], num_classes=len(label_map))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(images, labels, epochs=2000, batch_size=32, validation_split=0.2)

Loading image from: dataset\Andy\image_0.jpg
Loading image from: dataset\Andy\image_1.jpg
Loading image from: dataset\Andy\image_10.jpg
Loading image from: dataset\Andy\image_100.jpg
Loading image from: dataset\Andy\image_11.jpg
Loading image from: dataset\Andy\image_12.jpg
Loading image from: dataset\Andy\image_13.jpg
Loading image from: dataset\Andy\image_14.jpg
Loading image from: dataset\Andy\image_15.jpg
Loading image from: dataset\Andy\image_16.jpg
Loading image from: dataset\Andy\image_17.jpg
Loading image from: dataset\Andy\image_18.jpg
Loading image from: dataset\Andy\image_19.jpg
Loading image from: dataset\Andy\image_2.jpg
Loading image from: dataset\Andy\image_20.jpg
Loading image from: dataset\Andy\image_21.jpg
Loading image from: dataset\Andy\image_22.jpg
Loading image from: dataset\Andy\image_23.jpg
Loading image from: dataset\Andy\image_24.jpg
Loading image from: dataset\Andy\image_25.jpg
Loading image from: dataset\Andy\image_26.jpg
Loading image from: dataset\Andy\ima

C:\Users\RTX\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
C:\Users\RTX\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 399ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 327ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 332ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 325ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 334ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 330ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 325ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch

In [14]:
model.save('face_recognition_model.h5')
model = tf.keras.models.load_model('face_recognition_model.h5')

# Perform Face Recognition

In [19]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, 1.1, 4)
    return faces

In [20]:
def preprocess_image_array(image_array, target_size=(224, 224)):
    image = cv2.resize(image_array, target_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


In [21]:
def recognize_faces(image, model, label_map):
    faces = detect_faces(image)
    for (x, y, w, h) in faces:
        face_img = image[y:y+h, x:x+w]
        face_img = preprocess_image_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        
        prediction = model.predict(face_img)
        label_index = np.argmax(prediction)
        label_name = [name for name, index in label_map.items() if index == label_index][0]
        
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(image, label_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    return image


In [23]:
test_image = cv2.imread('image.jpg')
recognized_image = recognize_faces(test_image, model, label_map)
cv2.imshow('Recognized Faces', recognized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


# Detect in live stream

In [25]:
import cv2
import numpy as np
import tensorflow as tf

# Load your pre-trained model
model = tf.keras.models.load_model('face_recognition_model.h5')

# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Assuming label_map is already defined (e.g., loaded from a file or predefined)
# label_map = {'person1': 0, 'person2': 1, ...}

def preprocess_image_array(image_array, target_size=(224, 224)):
    image = cv2.resize(image_array, target_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def recognize_faces(image, model, label_map):
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        face_img = image[y:y+h, x:x+w]
        face_img = preprocess_image_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        
        prediction = model.predict(face_img)
        label_index = np.argmax(prediction)
        label_name = [name for name, index in label_map.items() if index == label_index][0]
        
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(image, label_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    return image

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()
    
    # Perform face recognition on the frame
    recognized_frame = recognize_faces(frame, model, label_map)
    
    # Display the resulting frame
    cv2.imshow('Live Face Recognition', recognized_frame)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━